# Coordinate da Indirizzo

In [2]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Novegro 20090 Segrate")
start_lat=location.latitude 
start_lng=location.longitude
print(start_lat,start_lng)


45.4688535 9.2776371


In [12]:
import geopy.geocoders
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Como 9 Barzago")
end_lat=location.latitude 
end_lng=location.longitude
print(end_lat,end_lng)

45.7578372 9.3097403


# Calcolo percorso più veloce

In [14]:
#Find Distance
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import IFrame
import folium
%matplotlib inline

np.random.seed(0)
ox.__version__

ox.settings.log_console=True
ox.settings.cache_folder='C:\\Users\\lucaf\\OneDrive\\Desktop\\Esercizi\\Cache'
place = "Milan,Lombardy,Italy"
mode="drive"
G = ox.graph_from_place(place, network_type=mode)
Gp = ox.project_graph(G)

KeyboardInterrupt: 

In [85]:
orig_point = (start_lat, start_lng) # UTM coordinates
dest_point = (end_lat, end_lng)

orig_node, dest_node = ox.nearest_nodes(
                            Gp,
                            X=[orig_point[0], dest_point[0]],
                            Y=[orig_point[1], dest_point[1]]
                                       )

In [86]:
orig_edge, dest_edge = ox.nearest_edges(
                            Gp,
                            X=[orig_point[0], dest_point[0]],
                            Y=[orig_point[1], dest_point[1]]
                                       )

In [87]:
print(orig_point,dest_point)
print(orig_node,dest_node)

(45.4688535, 9.2776371) (45.7578372, 9.3097403)
583043033 583043033


In [80]:
route = ox.shortest_path(Gp, orig_node, dest_node, weight="length")

In [13]:
shortest_route_map = ox.plot_route_folium(G, route,tiles='openstreetmap')
shortest_route_map

NameError: name 'ox' is not defined

In [ ]:

# see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

# same thing again, but this time pass in a few default speed values (km/hour)
# to fill in edges with missing `maxspeed` from OSM
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

# calculate two routes by minimizing travel distance vs travel time
orig = list(G)[1]
dest = list(G)[120]
route1 = ox.shortest_path(G, orig, dest, weight="length")
route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

# compare the two routes
route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "length")))
route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time")))
route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "travel_time")))
print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")


# Mappa Trasporti

In [2]:
import pandas as pd
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import IFrame
import folium
import pandas as pd
import osmnx as ox

ox.settings.cache_folder='C:\\Users\\lucaf\\OneDrive\\Desktop\\Esercizi\\Cache'


# Bus

In [4]:
places=["Milan,Lombardy,Italy","Segrate,Lombardy,Italy"]
tags = {"public_transport":"platform",
        "bus":"yes",
        "node":True}

all_pois = ox.geometries_from_place(places,tags)
bus=pd.DataFrame(all_pois)
bus=bus[["name","public_transport","geometry"]].reset_index()
bus=bus[bus.element_type=="node"]
bus["lat"]=bus.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
bus["lon"]=bus.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])
bus.to_csv("bus_location.csv")
"""
IconLayer
=========
"""
import pydeck as pdk
import pandas as pd


# Data from OpenStreetMap, accessed via osmpy

ICON_URL = "https://upload.wikimedia.org/wikipedia/commons/b/b3/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Bus_Stop_%E2%80%93_Transportation_%E2%80%93_Light.png"

icon_data = {

    "url": ICON_URL,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}

bus["lat"]=bus["lat"].apply(lambda x : float(x))
bus["lon"]=bus["lon"].apply(lambda x : float(x))

data=bus


data["icon_data"] = None
for i in data.index:
    data["icon_data"][i] = icon_data

data=data[["lat","lon","name","icon_data"]]


bus_layer = pdk.Layer(
    type="IconLayer",
    data=data,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)


C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\2522733735.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["icon_data"][i] = icon_data


# Metro e Treno

In [5]:
tags2 = {"public_transport":"station"}

all_pois2 = ox.geometries_from_place(places,tags2)
point2=pd.DataFrame(all_pois2).reset_index()
point2=point2[point2.element_type=="node"]
point2=point2[["name","public_transport","geometry","subway","railway"]].reset_index()

point2["lat"]=point2.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
point2["lon"]=point2.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])

train=point2[point2.subway!="yes"]
metro=point2[point2.subway=="yes"]

train.to_csv("train_location.csv")
metro.to_csv("metro_location.csv")

In [20]:
"""
IconLayer
=========
"""

import pydeck as pdk
import pandas as pd


# Data from OpenStreetMap, accessed via osmpy

ICON_URL = "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png"

icon_data = {

    "url": ICON_URL,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}

train["lat"]=train["lat"].apply(lambda x : float(x))
train["lon"]=train["lon"].apply(lambda x : float(x))

data=metro

data["icon_data"] = None
for i in data.index:
    data["icon_data"][i] = icon_data

data=data[["lat","lon","name","icon_data"]]


train_layer = pdk.Layer(
    type="IconLayer",
    data=data,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)

C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\1533603735.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["lat"]=train["lat"].apply(lambda x : float(x))
C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\1533603735.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["lon"]=train["lon"].apply(lambda x : float(x))
C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\1533603735.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [8]:
"""
IconLayer
=========
"""
import pydeck as pdk
import pandas as pd


# Data from OpenStreetMap, accessed via osmpy

ICON_URL = "https://upload.wikimedia.org/wikipedia/commons/5/57/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Subway_%E2%80%93_Transportation_%E2%80%93_Light.png"

icon_data = {

    "url": ICON_URL,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}

metro["lat"]=metro["lat"].apply(lambda x : float(x))
metro["lon"]=metro["lon"].apply(lambda x : float(x))

data=metro

data["icon_data"] = None
for i in data.index:
    data["icon_data"][i] = icon_data

data=data[["lat","lon","name","icon_data"]]


metro_layer = pdk.Layer(
    type="IconLayer",
    data=data,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)

C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\3496649935.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro["lat"]=metro["lat"].apply(lambda x : float(x))
C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\3496649935.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro["lon"]=metro["lon"].apply(lambda x : float(x))
C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\3496649935.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

# Parcheggi

In [9]:
tags3 = {"amenity":"parking"}

all_pois3 = ox.geometries_from_place(places,tags3)
parcheggi=pd.DataFrame(all_pois3).reset_index()
parcheggi_nodi=parcheggi[parcheggi.element_type=="node"]
parcheggi_way=parcheggi[parcheggi.element_type=="way"]
parcheggi_relation=parcheggi[parcheggi.element_type=="relation"]

In [10]:
parcheggi_nodi=parcheggi_nodi[["name","geometry"]]
parcheggi_nodi["lat"]=parcheggi_nodi.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
parcheggi_nodi["lon"]=parcheggi_nodi.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])

In [11]:
parcheggi_way=parcheggi_way[["name","geometry"]]
parcheggi_way["lat"]=parcheggi_way.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[0])
parcheggi_way["lon"]=parcheggi_way.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[1])

In [12]:
parcheggi_relation=parcheggi_relation[["name","geometry"]]
parcheggi_relation["split"]=parcheggi_relation.geometry.apply(lambda x:str(x).split("(")[0])
parcheggi_polygon=parcheggi_relation[parcheggi_relation.split=="POLYGON "]
parcheggi_multipolygon=parcheggi_relation[parcheggi_relation.split=="MULTIPOLYGON "]

In [13]:
parcheggi_polygon["lat"]=parcheggi_polygon.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[1])
parcheggi_polygon["lon"]=parcheggi_polygon.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[0])
parcheggi_polygon=parcheggi_polygon[["name","geometry","lat","lon"]]

C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\4072866983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parcheggi_polygon["lat"]=parcheggi_polygon.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[1])
C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\4072866983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parcheggi_polygon["lon"]=parcheggi_polygon.geometry.apply(lambda x:str(x).split("(")[2].split(",")[0].split(" ")[0])


In [14]:
parcheggi_multipolygon["lat"]=parcheggi_multipolygon.geometry.apply(lambda x:str(x).split("(")[3].split(",")[0].split(" ")[1])
parcheggi_multipolygon["lon"]=parcheggi_multipolygon.geometry.apply(lambda x:str(x).split("(")[3].split(",")[0].split(" ")[0])
parcheggi_multipolygon=parcheggi_multipolygon[["name","geometry","lat","lon"]]

C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\877092904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parcheggi_multipolygon["lat"]=parcheggi_multipolygon.geometry.apply(lambda x:str(x).split("(")[3].split(",")[0].split(" ")[1])
C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\877092904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parcheggi_multipolygon["lon"]=parcheggi_multipolygon.geometry.apply(lambda x:str(x).split("(")[3].split(",")[0].split(" ")[0])


In [24]:
parcheggi_complete=pd.concat([parcheggi_nodi, parcheggi_way,parcheggi_polygon,parcheggi_multipolygon], ignore_index=True)
parcheggi_complete["lat"]=parcheggi_complete["lat"].apply(lambda x : float(x))
parcheggi_complete["lon"]=parcheggi_complete["lon"].apply(lambda x : float(x))
parcheggi_complete.to_csv("parcheggi_location.csv")

In [16]:
"""
IconLayer
=========
"""
import pydeck as pdk
import pandas as pd


# Data from OpenStreetMap, accessed via osmpy

ICON_URL = "https://upload.wikimedia.org/wikipedia/commons/c/c7/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Parking_%E2%80%93_Transportation_%E2%80%93_Light.png"

icon_data = {

    "url": ICON_URL,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}

parcheggi_complete["lat"]=parcheggi_complete["lat"].apply(lambda x : float(x))
parcheggi_complete["lon"]=parcheggi_complete["lon"].apply(lambda x : float(x))

data=parcheggi_complete


data["icon_data"] = None
for i in data.index:
    data["icon_data"][i] = icon_data

data=data[["lat","lon","name","icon_data"]]


parking_layer = pdk.Layer(
    type="IconLayer",
    data=data,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)


C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\3532726547.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["icon_data"][i] = icon_data


# Origin

In [17]:
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
import pydeck as pdk
INITIAL_VIEW_STATE = pdk.ViewState(
  latitude=45.47185532715593, 
  longitude=9.275071955673953,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=0
)

In [21]:
"""
IconLayer
=========
"""
import pydeck as pdk
import pandas as pd


# Data from OpenStreetMap, accessed via osmpy

ICON_URL = "https://upload.wikimedia.org/wikipedia/commons/7/78/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Temple_%E2%80%93_Tourism_%E2%80%93_Light.png"

icon_data = {

    "url": ICON_URL,
    "width": 32,
    "height": 37,
    "anchorY": 'auto',
}


origin=pd.DataFrame({"lat":45.47185532715593,"lon":9.275071955673953,"name":"Esposizioni Novegro"}, index=[0])

origin["lat"]=origin["lat"].apply(lambda x : float(x))
origin["lon"]=origin["lon"].apply(lambda x : float(x))

data=origin

data["icon_data"] = None
for i in data.index:
    data["icon_data"][i] = icon_data

data=data[["lat","lon","name","icon_data"]]



view_state = INITIAL_VIEW_STATE

origin_layer = pdk.Layer(
    type="IconLayer",
    data=data,
    get_icon="icon_data",
    get_size=10,
    size_scale=3,
    get_position=["lon", "lat"],
    pickable=True,
)

C:\Users\lucaf\AppData\Local\Temp\ipykernel_2400\2543572718.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["icon_data"][i] = icon_data


In [23]:
pdk.Deck(layers=[parking_layer,train_layer,bus_layer,metro_layer,origin_layer], initial_view_state=view_state, tooltip={"text": "{name}"})

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 45.47185532715593,
    "longitude": 9.275071955673953,
    "maxZoom": 16,
    "pitch": 45,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "IconLayer",
      "data": [
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4641385,
          "lon": 9.1885548,
          "name": "Duomo"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4553979,
          "lon": 9.1129252,
          "name": "Bisceglie"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4368888,
          "lon": 9.1681331,
          "name": "Famagosta"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4335776,
          "lon": 9.2380577,
          "name": "Rogoredo FS"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4643154,
          "lon": 9.2363556,
          "name": "Stazione Forlanini"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4620064,
          "lon": 9.2403893,
          "name": "Repetti"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.429086,
          "lon": 9.2563154,
          "name": "San Donato"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4599554,
          "lon": 9.1893681,
          "name": "Missori"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.org/wikipedia/commons/1/16/Map_marker_icon_%E2%80%93_Nicolas_Mollet_%E2%80%93_Train_%E2%80%93_Transportation_%E2%80%93_Light.png",
            "width": 32
          },
          "lat": 45.4808531,
          "lon": 9.2252588,
          "name": "Piola"
        },
        {
          "icon_data": {
            "anchorY": "auto",
            "height": 37,
            "url": "https://upload.wikimedia.o